In [1]:
from dash import Dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import datetime, date


In [2]:
from os import pardir, path, getcwd, path
import sys

# mod_path = path.abspath(path.join(pardir))
pack_global = r'/Users/karimkhalil/Coding/development'
pack_project = path.abspath(path.join(getcwd(), pardir))

dirs = [pack_global,pack_project]


for i in dirs:
    # print(i)
    if i not in sys.path:
        sys.path.append(i)

print(sys.path)

['/Users/karimkhalil/Coding/development/stocksapp/notebooks', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python39.zip', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development', '/Users/karimkhalil/Coding/development/stocksapp']


In [3]:
# from mypackages.db_loader.mongo import DB_mongo

from mypackages import DB_mongo
from project_package import get_dropdown_label
from pymongo import MongoClient, ASCENDING , DESCENDING
import pandas as pd

['admin', 'config', 'db_fin', 'local']
mongo imported from mypackages.db_loader.mongo
['/Users/karimkhalil/Coding/development/stocksapp/notebooks', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python39.zip', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/stocksapp/stocksappenv/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development', '/Users/karimkhalil/Coding/development/stocksapp']
project package imported


In [4]:
stock = DB_mongo._CLIENT.db_fin
mongo_stock = DB_mongo(stock)
mongo_stock.db.tables

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_fin'), 'tables')

In [5]:
tables = mongo_stock.all_tables
print(tables)

['cf', 'prices_bckup', 'company_today', 'mrktcap', 'company_wfinancials', 'prices_n_fin', 'bs', 'fx', 'profile', 'idx_price', 'prices', 'idx_all', 'pnl']


In [6]:
mongo_stock.get_df(mongo_stock.db.idx_all)

,_id,symbol,name,currency,stockExchange,exchangeShortName
0,62cc22fa4c9eba2a564feccc,TYX,Treasury Yield 30 Years,USD,NYBOT,INDEX
1,62cc22fa4c9eba2a564feccd,FVX,Treasury Yield 5 Years,USD,NYBOT,INDEX
2,62cc22fa4c9eba2a564fecce,TNX,Treasury Yield 10 Years,USD,NYBOT,INDEX
3,62cc22fa4c9eba2a564feccf,SPGSCI,S&P GSCI Index,USD,SNP,INDEX
4,62cc22fa4c9eba2a564fecd0,STI,STI Index,SGD,SES,INDEX
...,...,...,...,...,...,...
58,62cc22fa4c9eba2a564fed06,XAX,NYSE AMEX COMPOSITE INDEX,USD,New York Stock Exchange,INDEX
59,62cc22fa4c9eba2a564fed07,MID,S&P MID CAP 400 INDEX,USD,New York Stock Exchange,INDEX
60,62cc22fa4c9eba2a564fed08,NYA,NYSE COMPOSITE (DJ),USD,New York Stock Exchange,INDEX
61,62cc22fa4c9eba2a564fed09,X60.TS,S&P/TSX 60 Index,CAD,Toronto,INDEX


In [7]:
mongo_stock.get_df(mongo_stock.db.profile, Symbol='MSFT')

,_id,Symbol,Price,Beta,VolAvg,MktCap,LastDiv,Range,Changes,companyName,...,zip,DCF_diff,DCF,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isFund,isAdr
0,629c9ee60206e9ef45214eca,MSFT,270.02,0.93616,33000374,2.019488e+12,2.42,246.44-349.67,-4.56,Microsoft Corporation,...,98052-6399,1.54418,271.564,https://financialmodelingprep.com/image-stock/...,1986-03-13,False,False,True,False,False


In [8]:
df_idx_all = mongo_stock.get_df(mongo_stock.db.idx_all)
df_idx_price = mongo_stock.get_df(mongo_stock.db.idx_price)
df_idx_price = df_idx_price.merge(df_idx_all[['symbol', 'name']])
idx_sym = mongo_stock.get_unique(mongo_stock.db.idx_price, 'symbol')
# idx_sym = mongo_stock.db.idx_price.distinct('symbol')
df_idx = df_idx_all.loc[df_idx_all['symbol'].isin(idx_sym), ['symbol', 'name' , 'currency', 'stockExchange']]
df_idx[['symbol', 'name']].to_dict('records')
market_idx_dropdown = [{'label': i['name'], 'value': i['symbol']} for i in df_idx[['symbol', 'name']].to_dict('records') ]    
df_idx_info = df_idx_all.loc[df_idx_all['symbol'].isin(idx_sym), ['symbol', 'name']]
fig = px.line(df_idx_price, x='datetime', y = 'adjClose', color='name')

In [10]:
# def get_dropdown_label(some):
#     for i,j in enumerate(market_idx_dropdown):
#         if j['value'] == some:
#             # print(i)
#             index = i
#     # print(index)
#     return market_idx_dropdown[index]['label']

# get_dropdown_label('TYX')

In [11]:
get_dropdown_label(market_idx_dropdown, 'TYX')

'Treasury Yield 30 Years'

In [30]:
df_idx_option = df_idx[df_idx['symbol']=='TYX']
df_idx_option.columns= ['Symbol', 'Name', 'Currency', 'Stock Exchange']
df_idx_option = df_idx_option.transpose().reset_index()
df_idx_option.columns = ["item", "value"]
df_idx_option


,item,value
0,Symbol,TYX
1,Name,Treasury Yield 30 Years
2,Currency,USD
3,Stock Exchange,NYBOT


In [ ]:
df_date_check = df_idx_price
df_date_check['datetime'] = df_date_check['datetime'].dt.strftime("%Y-%m-%d")

In [ ]:
df_date_check['datetime'] = pd.to_datetime(df_date_check['datetime'])
df_date_check.loc[df_date_check['datetime'].between( "2020-08-30", "2021-08-30")]
# df_date_check.loc[df_date_check['datetime']> "2021-08-30"]

In [ ]:
df_date_check.info()

In [ ]:
d = df_idx_price.to_dict('records')

In [ ]:
for i in d[0]:
    print(i, d[0][i])

In [ ]:
df_fig = df_idx_price
date_min_str = pd.to_datetime(df_fig['datetime'].idxmin()).strftime("%m/%d/%Y")
date_max_str = pd.to_datetime(df_fig['datetime'].idxmax()).strftime("%m/%d/%Y")

date_min = pd.to_datetime(df_fig['datetime'].idxmin()).strftime("%m/%d/%Y")
date_max = pd.to_datetime(df_fig['datetime'].idxmax()).strftime("%m/%d/%Y")

In [ ]:
df_idx_price.date

In [ ]:
df_idx_price['datetime'].dt.strftime("%m-%d-%Y")

In [ ]:
df_idx_price.info()

In [ ]:
df_idx_price['date'].apply(lambda x: x.isoformat())

In [ ]:
cols = [i for i in df_idx_all.columns if i != '_id']

In [ ]:
d =df_fig.to_dict('records')[0]

In [ ]:
type(d['date'])

In [ ]:
pd.to_datetime(df_idx_price['datetime'].idxmax()).strftime("%m/%d/%Y")

In [ ]:
datetime(1995, 8, 5)

In [ ]:
datetime.strptime('2010-01-01', '%Y-%m-%d')

In [ ]:
datetime.strftime(datetime(1995, 8, 5), '%Y-%m-%d')

In [ ]:
type(df_idx_price.to_dict('records'))